Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [19]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./whiskey-reviews-dspt4/train.csv')
test = pd.read_csv('./whiskey-reviews-dspt4/test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [9]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133",1
1,3861,"\nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)",0
2,655,"\nThis release is a port version of Amrut’s Intermediate Sherry — a sort of port pipe sandwich. The spirit is matured in both unused casks and bourbon casks, then spends a few months in port pipes, and then returns to bourbon casks. The result is a Pink Floyd show of a whisky: vibrant, colorful, complex, and nearly too much. A blackcurrant and wispy, smoky nose gives way to an intense and bittersweet mix of chili, blackcurrant, oak, damson, dark chocolate, and peat. Astounding.",1
3,555,"\nThis 41 year old single cask was aged in a sherry butt and interacted magnificently with that wood. The nose offers peeled red apples, sultanas, honey, fudge, milk chocolate, and American cream soda. Balanced and harmonious throughout, the palate yields remarkably fresh orchard fruits, pipe tobacco, black pepper, and ginger. Long in the finish, with cinnamon drying to dark chocolate, and slightly smoky, unobtrusively tannic oak. A 1970s classic!",1
4,1965,"\nQuite herbal on the nose, with aromas of dried tarragon, parsley, and dill, along with cherry, milk chocolate, orange blossom, vanilla bean, and sweet tea. The palate is slightly grain-forward, but is full and well balanced, with orange, dried flowers, chamomile tea, lemon rind, and ginger. The finish offers cinnamon and milk chocolate against a note of candied lemon. Youthful and barrel-influenced, but admirably executed.",1


In [10]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts()

1    2881
0    1141
2    65  
Name: ratingCategory, dtype: int64

In [11]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
1398,5018,"\nThe first thing out of the glass is bourbon — 80 proof bottlings will do that — followed by a secondary note of strong-brewed tea with a dollop of honey. It's quite sweet, and hot, and the tea largely disappears except as a frill around the edges. This is more a honey drink than a tea drink, and there are better honey drinks out there.",0
3829,3753,"\nWhat you look for in a finished whisky is for some of the distillery's character to be apparent, but also enhanced. Benrinnes is a big, meaty dram, which suits sherry and its feral aspect is there. Think of roast beef with damson jam, date, and blackberry. The palate is thick with licorice sweetness and a bitter edge. It has power, and while you can see the join between original spirit and finish, it’s a pretty happy match. £60/500 ml",0
2902,4516,"\nMuch more oak influence than its younger sibling. That lovely sweetness of the 19 year old Experimental Collection Giant French Oak Barrel bottling is there at the beginning, but it’s quickly overtaken by oak spice, leather, and tannin notes. This one was left in the barrel a bit longer than I would prefer. If you like a wood-driven whiskey, then consider trying it. But if it’s balance you desire, look elsewhere. Price is per 375 ml.",0


In [12]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
1247,5047,"\nFreshly cut oak and chewing tobacco to start, followed by hints of green apple, wood chips, and a warm, wet sharpening stone. On the palate, the wood heightens and becomes extremely bitter. This whiskey’s saving grace is that it’s extremely palatable in cocktails.",2
3289,5083,"\nThere are flavored—cinnamon and coconut—versions at lower proof; this is the “Original,” unflavored at full proof. Unaged and clear, the aroma is clean and full of roasted corn and a ripple of deep caramel. Surprisingly appealing, actually. Tastes like raw new make, though: green, feinty, vegetal, and crying out for cola, ginger ale, even Mountain Dew to help mask it. Are we done with ‘moonshine’ yet? Please?",2
3890,5032,"\nCitrus peel, light maple syrup, and almonds, with emerging grape and vanilla. Somewhat elegant in nature, but the flavors do not especially complement each other. (Exclusive to Binny’s Beverage Depot.) \r\n",2


### Split the Training Set into Train/Validation

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


### Define Pipeline Components

In [20]:
from sklearn.pipeline import Pipeline

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('rfc', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [22]:
X_train.head()

1782    \nVery fragrant, with notes of fresh ripe plum, crisp spice, herbs, caramel, treacle, black pepper, and hints of dark chocolate. Leave the glass alone and the spices really hit their stride. The flavor shifts imperceptibly, covering citrus, vanilla, chocolate, and ripe orchard fruits, generating a slow build-up of spices like a gathering storm: your taste buds have nowhere left to run. Beautifully paced.                                                                             
3414    \nThe good news: This is one of the best Highland Park whiskies I have ever tasted. The bad news: it’s the new release for Travel Retail (formerly Duty Free). It’s lush, well-balanced, and very complex. Well-defined notes of toffee, candied fruit, and roasted nuts are accentuated by background honey, chocolate mousse, and smoke. Here’s the icing on the cake: it’s bottled at 47.5%, which really allows the flavors to shine. If you’re going overseas, consider tracking down a bottle.
2744    \nA sw

In [26]:
parameters = {
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (2, 5, 10),
    'rfc__n_estimators': (100, 500),
    'rfc__max_depth': (10, 20)
}

grid_search = GridSearchCV(pipe, parameters, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  4.1min finished


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [27]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7243822440618519
{'rfc__max_depth': 20, 'rfc__n_estimators': 500, 'vect__max_df': 1.0, 'vect__min_df': 10}


### TruncatedSVD

In [28]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
score = grid_search.predict(X_test)
accuracy_score(score, y_test)

0.7249388753056235

In [35]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD

# Use parameters from GridSearch in previous section
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=10, 
                       max_df=1.0)

# Add dimensionality reduction
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

# Use parameters from RandomSearch in previous section
rfc = RandomForestClassifier(n_estimators=500,
                             max_depth=20,
                             random_state=42)

In [36]:
params = {
    'svd__n_components': stats.randint(100, 500),
}

In [37]:
pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', rfc)         # RandomForest Classifier
])

In [38]:
# Fit
random_search = RandomizedSearchCV(pipe, params, cv=2, n_iter=3, n_jobs=-1, verbose=1)
random_search.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.2min finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=10,
                                                       

In [40]:
print(random_search.best_score_)
print(random_search.best_params_)

0.7121444533030892
{'svd__n_components': 113}


In [41]:
# Evaluate on test data
score = random_search.predict(X_test)
accuracy_score(score, y_test)

0.7212713936430318

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [29]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [30]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [31]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [32]:
subNumber = 0

In [33]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
lsi = ...
vect = ...
clf = ...

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?